In [84]:
import numpy as np
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


C:\Users\User\Anaconda2\lib\site-packages\IPython\core\magics\pylab.py:161: UserWarning: pylab import has clobbered these variables: ['dist']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [85]:
data = pd.read_csv('checkins.dat', delimiter='|', skipinitialspace= True).dropna(axis = 0)

In [86]:
data.head(5)

,id,user_id,venue_id,latitude,longitude,created_at
2,984222,15824.0,5222.0,38.895112,-77.036366,2012-04-21 17:43:47
4,984234,44652.0,5222.0,33.800745,-84.410520,2012-04-21 17:43:43
8,984291,105054.0,5222.0,45.523452,-122.676207,2012-04-21 17:39:22
10,984318,2146539.0,5222.0,40.764462,-111.904565,2012-04-21 17:35:46
11,984232,93870.0,380645.0,33.448377,-112.074037,2012-04-21 17:38:18


In [87]:
print data.shape
data.columns

(396634, 6)


Index([u'id    ', u'user_id ', u'venue_id ', u'latitude      ',
       u'longitude     ', u'created_at      '],
      dtype='object')

In [88]:
print type(data)
cols = [col.strip() for col in data.columns]
print cols
data.columns = cols


<class 'pandas.core.frame.DataFrame'>
['id', 'user_id', 'venue_id', 'latitude', 'longitude', 'created_at']


In [89]:
from sklearn.cluster import MeanShift

In [90]:
X = data[['latitude', 'longitude']]
X.head()

,latitude,longitude
2,38.895112,-77.036366
4,33.800745,-84.410520
8,45.523452,-122.676207
10,40.764462,-111.904565
11,33.448377,-112.074037


In [91]:
mean_shift = MeanShift(bandwidth=0.1)
mean_shift.fit(X[:100000])

MeanShift(bandwidth=0.1, bin_seeding=False, cluster_all=True, min_bin_freq=1,
     n_jobs=1, seeds=None)

In [92]:
labels = mean_shift.labels_
centers = mean_shift.cluster_centers_

In [93]:
unique_labels = np.unique(labels)
number_of_clusters = len(unique_labels)

In [94]:
print number_of_clusters, len(centers), len(unique_labels), len(labels)
print centers

3230 3230 3230 100000
[[  40.7177164   -73.99183542]
 [  33.44943805 -112.00213969]
 [  33.44638027 -111.90188756]
 ..., 
 [  46.7323875  -117.0001651 ]
 [  29.6899563   -95.8996757 ]
 [  31.3787916   -95.3213317 ]]


In [95]:
count_cluster_elements = dict()
for i in range(len(labels)):
    if count_cluster_elements.has_key(labels[i]):
        count_cluster_elements[labels[i]] += 1
    else:
        count_cluster_elements[labels[i]] = 1
informative_labels = [label for label in unique_labels if count_cluster_elements[label] > 15]
informative_centers = [centers[i] for i in range(len(centers)) if informative_labels.count(labels[i]) > 0]

In [96]:
print len(informative_labels)


593


In [97]:
offices = []
offices.append([33.751277, -118.188740])
offices.append([25.867736, -80.324116])
offices.append([51.503016, -0.075479])
offices.append([52.378894, 4.885084])
offices.append([39.366487, 117.036146])
offices.append([-33.868457, 151.205134])

In [98]:
distances = []
for center in informative_centers:
    min_dist = 1000000
    for office in offices:
        dist = np.sqrt((center[0] - office[0])**2 + (center[1] - office[1])**2)
        if dist < min_dist:
            min_dist = dist
    distances.append(min_dist)
distances = np.array(distances)
dist_ind = np.argsort(distances)            

In [99]:
print dist_ind
print distances

[ 371  334  365 ..., 1004 2525 1651]
[ 16.14371999   6.19395917   6.29424146 ...,  21.63139086  13.03541101
  16.03768774]


In [100]:
distances[dist_ind]

array([  7.83475816e-03,   9.35331619e-03,   2.26740662e-02, ...,
         8.73616530e+01,   8.73673961e+01,   8.75744820e+01])

In [101]:
informative_centers = np.array(informative_centers)

In [102]:
print informative_centers[371]
answer = informative_centers[371]

[ -33.86063043  151.20477593]


In [105]:
def write_answer(answers, filename):
    with open(filename, "w") as fout:
        fout.write(" ".join([str(num) for num in answers]))

In [106]:
write_answer(answer, "week1.txt")